# 4 - Convolutional Sentiment Analysis

In the previous notebooks, we managed to achieve a test accuracy of ~85% using RNNs and an implementation of the [Bag of Tricks for Efficient Text Classification](https://arxiv.org/abs/1607.01759) model. In this notebook, we will be using a *convolutional neural network* (CNN) to conduct sentiment analysis, implementing the model from [Convolutional Neural Networks for Sentence Classification](https://arxiv.org/abs/1408.5882).

**Note**: This tutorial is not aiming to give a comprehensive introduction and explanation of CNNs. For a better and more in-depth explanation check out [here](https://ujjwalkarn.me/2016/08/11/intuitive-explanation-convnets/) and [here](https://cs231n.github.io/convolutional-networks/).

Traditionally, CNNs are used to analyse images and are made up of one or more *convolutional* layers, followed by one or more linear layers. The convolutional layers use filters (also called *kernels* or *receptive fields*) which scan across an image and produce a processed version of the image. This processed version of the image can be fed into another convolutional layer or a linear layer. Each filter has a shape, e.g. a 3x3 filter covers a 3 pixel wide and 3 pixel high area of the image, and each element of the filter has a weight associated with it, the 3x3 filter would have 9 weights. In traditional image processing these weights were specified by hand by engineers, however the main advantage of the convolutional layers in neural networks is that these weights are learned via backpropagation. 

The intuitive idea behind learning the weights is that your convolutional layers act like *feature extractors*, extracting parts of the image that are most important for your CNN's goal, e.g. if using a CNN to detect faces in an image, the CNN may be looking for features such as the existance of a nose, mouth or a pair of eyes in the image.

So why use CNNs on text? In the same way that a 3x3 filter can look over a patch of an image, a 1x2 filter can look over a 2 sequential words in a piece of text, i.e. a bi-gram. In the previous tutorial we looked at the FastText model which used bi-grams by explicitly adding them to the end of a text, in this CNN model we will instead use multiple filters of different sizes which will look at the bi-grams (a 1x2 filter), tri-grams (a 1x3 filter) and/or n-grams (a 1x$n$ filter) within the text.

The intuition here is that the appearance of certain bi-grams, tri-grams and n-grams within the review will be a good indication of the final sentiment.

## Preparing Data

As in the previous notebooks, we'll prepare the data. 

Unlike the previous notebook with the FastText model, we no longer explicitly need to create the bi-grams and append them to the end of the sentence.

As convolutional layers expect the batch dimension to be first we can tell TorchText to return the data already permuted using the `batch_first = True` argument on the field.

In [1]:
import torch
from torchtext import data
from torchtext import datasets
from torchtext import vocab
import random
import numpy as np

SEED = 1234

random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

TEXT = data.Field(tokenize = 'spacy', batch_first = True)
LABEL = data.LabelField(dtype = torch.float)

train_data, test_data = datasets.IMDB.splits(TEXT, LABEL, root='~/IDS576/data/')

train_data, valid_data = train_data.split(random_state = random.seed(SEED))

Build the vocab and load the pre-trained word embeddings.

In [2]:
MAX_VOCAB_SIZE = 25_000

glove_path = '/home/team3/IDS576/glove/'
glove_vectors = vocab.Vectors('glove.6B.100d.txt', glove_path)

TEXT.build_vocab(train_data, 
                 max_size = MAX_VOCAB_SIZE, 
                 vectors = glove_vectors, 
                 unk_init = torch.Tensor.normal_)

LABEL.build_vocab(train_data)

As before, we create the iterators.

In [139]:
BATCH_SIZE = 64

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

train_iterator, valid_iterator, test_iterator = data.BucketIterator.splits(
    (train_data, valid_data, test_data), 
    batch_size = BATCH_SIZE, 
    device = device)

## Build the Model

Now to build our model.

The first major hurdle is visualizing how CNNs are used for text. Images are typically 2 dimensional (we'll ignore the fact that there is a third "colour" dimension for now) whereas text is 1 dimensional. However, we know that the first step in almost all of our previous tutorials (and pretty much all NLP pipelines) is converting the words into word embeddings. This is how we can visualize our words in 2 dimensions, each word along one axis and the elements of vectors aross the other dimension. Consider the 2 dimensional representation of the embedded sentence below:

![](assets/sentiment9.png)

We can then use a filter that is **[n x emb_dim]**. This will cover $n$ sequential words entirely, as their width will be `emb_dim` dimensions. Consider the image below, with our word vectors are represented in green. Here we have 4 words with 5 dimensional embeddings, creating a [4x5] "image" tensor. A filter that covers two words at a time (i.e. bi-grams) will be **[2x5]** filter, shown in yellow, and each element of the filter with have a _weight_ associated with it. The output of this filter (shown in red) will be a single real number that is the weighted sum of all elements covered by the filter.

![](assets/sentiment12.png)

The filter then moves "down" the image (or across the sentence) to cover the next bi-gram and another output (weighted sum) is calculated. 

![](assets/sentiment13.png)

Finally, the filter moves down again and the final output for this filter is calculated.

![](assets/sentiment14.png)

In our case (and in the general case where the width of the filter equals the width of the "image"), our output will be a vector with number of elements equal to the height of the image (or lenth of the word) minus the height of the filter plus one, $4-2+1=3$ in this case.

This example showed how to calculate the output of one filter. Our model (and pretty much all CNNs) will have lots of these filters. The idea is that each filter will learn a different feature to extract. In the above example, we are hoping each of the **[2 x emb_dim]** filters will be looking for the occurence of different bi-grams. 

In our model, we will also have different sizes of filters, heights of 3, 4 and 5, with 100 of each of them. The intuition is that we will be looking for the occurence of different tri-grams, 4-grams and 5-grams that are relevant for analysing sentiment of movie reviews.

The next step in our model is to use *pooling* (specifically *max pooling*) on the output of the convolutional layers. This is similar to the FastText model where we performed the average over each of the word vectors, implemented by the `F.avg_pool2d` function, however instead of taking the average over a dimension, we are taking the maximum value over a dimension. Below an example of taking the maximum value (0.9) from the output of the convolutional layer on the example sentence (not shown is the activation function applied to the output of the convolutions).

![](assets/sentiment15.png)

The idea here is that the maximum value is the "most important" feature for determining the sentiment of the review, which corresponds to the "most important" n-gram within the review. How do we know what the "most important" n-gram is? Luckily, we don't have to! Through backpropagation, the weights of the filters are changed so that whenever certain n-grams that are highly indicative of the sentiment are seen, the output of the filter is a "high" value. This "high" value then passes through the max pooling layer if it is the maximum value in the output. 

As our model has 100 filters of 3 different sizes, that means we have 300 different n-grams the model thinks are important. We concatenate these together into a single vector and pass them through a linear layer to predict the sentiment. We can think of the weights of this linear layer as "weighting up the evidence" from each of the 300 n-grams and making a final decision. 

### Implementation Details

We implement the convolutional layers with `nn.Conv2d`. The `in_channels` argument is the number of "channels" in your image going into the convolutional layer. In actual images this is usually 3 (one channel for each of the red, blue and green channels), however when using text we only have a single channel, the text itself. The `out_channels` is the number of filters and the `kernel_size` is the size of the filters. Each of our `kernel_size`s is going to be **[n x emb_dim]** where $n$ is the size of the n-grams.

In PyTorch, RNNs want the input with the batch dimension second, whereas CNNs want the batch dimension first - we do not have to permute the data here as we have already set `batch_first = True` in our `TEXT` field. We then pass the sentence through an embedding layer to get our embeddings. The second dimension of the input into a `nn.Conv2d` layer must be the channel dimension. As text technically does not have a channel dimension, we `unsqueeze` our tensor to create one. This matches with our `in_channels=1` in the initialization of our convolutional layers. 

We then pass the tensors through the convolutional and pooling layers, using the `ReLU` activation function after the convolutional layers. Another nice feature of the pooling layers is that they handle sentences of different lengths. The size of the output of the convolutional layer is dependent on the size of the input to it, and different batches contain sentences of different lengths. Without the max pooling layer the input to our linear layer would depend on the size of the input sentence (not what we want). One option to rectify this would be to trim/pad all sentences to the same length, however with the max pooling layer we always know the input to the linear layer will be the total number of filters. **Note**: there an exception to this if your sentence(s) are shorter than the largest filter used. You will then have to pad your sentences to the length of the largest filter. In the IMDb data there are no reviews only 5 words long so we don't have to worry about that, but you will if you are using your own data.

Finally, we perform dropout on the concatenated filter outputs and then pass them through a linear layer to make our predictions.

Currently the `CNN` model can only use 3 different sized filters, but we can actually improve the code of our model to make it more generic and take any number of filters.

We do this by placing all of our convolutional layers in a  `nn.ModuleList`, a function used to hold a list of PyTorch `nn.Module`s. If we simply used a standard Python list, the modules within the list cannot be "seen" by any modules outside the list which will cause us some errors.

We can now pass an arbitrary sized list of filter sizes and the list comprehension will create a convolutional layer for each of them. Then, in the `forward` method we iterate through the list applying each convolutional layer to get a list of convolutional outputs, which we also feed through the max pooling in a list comprehension before concatenating together and passing through the dropout and linear layers.

In [177]:
class CNN(nn.Module):
    def __init__(self, rfn, vocab_size, embedding_dim, n_filters, filter_sizes, output_dim, 
                 dropout, pad_idx):
        
        super().__init__()
        
        self.rfn = rfn
                
        self.embedding = nn.Embedding(vocab_size, embedding_dim, padding_idx = pad_idx)
        
        self.convs = nn.ModuleList([
                                    nn.Conv2d(in_channels = 1, 
                                              out_channels = n_filters, 
                                              kernel_size = (fs, embedding_dim)) 
                                    for fs in filter_sizes
                                    ])
        
        self.fc = nn.Linear(len(filter_sizes) * n_filters, output_dim)
        
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, text):
        #text = [batch size, sent len]
        text = self.rfn(text)
        
        embedded = self.embedding(text)
                
        #embedded = [batch size, sent len, emb dim]
        
        embedded = embedded.unsqueeze(1)
        
        #embedded = [batch size, 1, sent len, emb dim]
        
        conved = [F.relu(conv(embedded)).squeeze(3) for conv in self.convs]
            
        #conved_n = [batch size, n_filters, sent len - filter_sizes[n] + 1]
                
        pooled = [F.max_pool1d(conv, conv.shape[2]).squeeze(2) for conv in conved]
        
        #pooled_n = [batch size, n_filters]
        
        cat = self.dropout(torch.cat(pooled, dim = 1))

        #cat = [batch size, n_filters * len(filter_sizes)]
            
        return self.fc(cat)

In [324]:
import torch.nn as nn
import torch
import numpy as np

class RFN(nn.Module):
    '''Wrapper class that can be inserted into a nn.Module based model to add an RFN layer'''
    
    
    def __init__(self, mu=.1, sigma=.05, rfn_flag=True):
        super(RFN, self).__init__()
        self.mu = mu
        self.sigma = sigma
        self.rfn_flag = rfn_flag
        self.shape_vals = None

    def forward(self, x):
        #print(x.size())
        if self.rfn_flag:
            print('x', x.dtype)
            print('rfn_x', self.random_masks(x).dtype)
            x = self.random_masks(x) * x
            x = x.long()
        return x
    
    
    def random_masks(self, x):
        self.shape_vals = x.size(), x[0].size(), x[0].nelement()
        #print(self.shape_vals)

        # Create ones array of size (batch,channel,horiz,vert)
        masks = np.ones(self.shape_vals[0])

        # Create random masks for each sample in the batch
        for mask in masks:
            zero_cnt = np.absolute(int(np.ceil((self.sigma * np.random.randn() + self.mu) * self.shape_vals[2])))
            zero_idx = np.random.choice(self.shape_vals[2], zero_cnt, replace=False)
            mask.reshape(self.shape_vals[2])[zero_idx] = 0
            mask.reshape(self.shape_vals[1])
        return torch.tensor(masks, dtype=torch.float, device=device)

We can also implement the above model using 1-dimensional convolutional layers, where the embedding dimension is the "depth" of the filter and the number of tokens in the sentence is the width.

We'll run our tests in this notebook using the 2-dimensional convolutional model, but leave the implementation for the 1-dimensional model below for anyone interested. 

We create an instance of our `CNN` class. 

We can change `CNN` to `CNN1d` if we want to run the 1-dimensional convolutional model, noting that both models give almost identical results.

In [179]:
INPUT_DIM = len(TEXT.vocab)
EMBEDDING_DIM = 100
N_FILTERS = 100
FILTER_SIZES = [3,4,5]
OUTPUT_DIM = 1
DROPOUT = 0.5
PAD_IDX = TEXT.vocab.stoi[TEXT.pad_token]

rfn = RFN(mu=.1, sigma=.05, rfn_flag=True)
rfn.to(device)

model = CNN(rfn, INPUT_DIM, EMBEDDING_DIM, N_FILTERS, FILTER_SIZES, OUTPUT_DIM, DROPOUT, PAD_IDX)

In [ ]:
INPUT_DIM = len(TEXT.vocab)
EMBEDDING_DIM = 100
N_FILTERS = 100
FILTER_SIZES = [3,4,5]
OUTPUT_DIM = 1
DROPOUT = 0.5
PAD_IDX = TEXT.vocab.stoi[TEXT.pad_token]

rfn = RFN(mu=.1, sigma=.05, rfn_flag=True)
rfn.to(device)

model = CNN(rfn, INPUT_DIM, EMBEDDING_DIM, N_FILTERS, FILTER_SIZES, OUTPUT_DIM, DROPOUT, PAD_IDX)

pretrained_embeddings = TEXT.vocab.vectors

model.embedding.weight.data.copy_(pretrained_embeddings)

UNK_IDX = TEXT.vocab.stoi[TEXT.unk_token]

model.embedding.weight.data[UNK_IDX] = torch.zeros(EMBEDDING_DIM)
model.embedding.weight.data[PAD_IDX] = torch.zeros(EMBEDDING_DIM)

import torch.optim as optim

optimizer = optim.Adam(model.parameters())

criterion = nn.BCEWithLogitsLoss()

model = model.to(device)
criterion = criterion.to(device)

In [180]:
INPUT_DIM

25002

Checking the number of parameters in our model we can see it has about the same as the FastText model. 

Both the `CNN` and the `CNN1d` models have the exact same number of parameters.

In [181]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 2,620,801 trainable parameters


Next, we'll load the pre-trained embeddings

In [182]:
pretrained_embeddings = TEXT.vocab.vectors

model.embedding.weight.data.copy_(pretrained_embeddings)

tensor([[ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [-0.0382, -0.2449,  0.7281,  ..., -0.1459,  0.8278,  0.2706],
        ...,
        [-0.1983, -0.2634,  0.4227,  ...,  0.1574, -0.3458,  0.4537],
        [-0.0640,  0.6652, -0.4537,  ..., -0.4337, -0.7348, -0.2443],
        [-0.2821,  0.0417,  0.4807,  ..., -0.5425, -0.7024,  1.3024]])

Then zero the initial weights of the unknown and padding tokens.

In [183]:
UNK_IDX = TEXT.vocab.stoi[TEXT.unk_token]

model.embedding.weight.data[UNK_IDX] = torch.zeros(EMBEDDING_DIM)
model.embedding.weight.data[PAD_IDX] = torch.zeros(EMBEDDING_DIM)

## Train the Model

Training is the same as before. We initialize the optimizer, loss function (criterion) and place the model and criterion on the GPU (if available)

In [184]:
import torch.optim as optim

optimizer = optim.Adam(model.parameters())

criterion = nn.BCEWithLogitsLoss()

model = model.to(device)
criterion = criterion.to(device)

We implement the function to calculate accuracy...

In [185]:
def binary_accuracy(preds, y):
    """
    Returns accuracy per batch, i.e. if you get 8/10 right, this returns 0.8, NOT 8
    """

    #round predictions to the closest integer
    rounded_preds = torch.round(torch.sigmoid(preds))
    correct = (rounded_preds == y).float() #convert into float for division 
    acc = correct.sum() / len(correct)
    return acc

We define a function for training our model...

**Note**: as we are using dropout again, we must remember to use `model.train()` to ensure the dropout is "turned on" while training.

In [186]:
def train(model, iterator, optimizer, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.train()
    
    for batch in iterator:
        
        optimizer.zero_grad()
        
        predictions = model(batch.text).squeeze(1)
        
        loss = criterion(predictions, batch.label)
        
        acc = binary_accuracy(predictions, batch.label)
        
        loss.backward()
        
        optimizer.step()
        
        epoch_loss += loss.item()
        epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

We define a function for testing our model...

**Note**: again, as we are now using dropout, we must remember to use `model.eval()` to ensure the dropout is "turned off" while evaluating.

In [187]:
def evaluate(model, iterator, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.eval()
    
    with torch.no_grad():
    
        for batch in iterator:

            predictions = model(batch.text).squeeze(1)
            
            loss = criterion(predictions, batch.label)
            
            acc = binary_accuracy(predictions, batch.label)

            epoch_loss += loss.item()
            epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

Let's define our function to tell us how long epochs take.

In [188]:
import time

def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [200]:
def load_model(mu, rfn_flag):
    INPUT_DIM = len(TEXT.vocab)
    EMBEDDING_DIM = 100
    N_FILTERS = 100
    FILTER_SIZES = [3,4,5]
    OUTPUT_DIM = 1
    DROPOUT = 0.5
    PAD_IDX = TEXT.vocab.stoi[TEXT.pad_token]

    rfn = RFN(mu=mu, sigma=.05, rfn_flag=rfn_flag)
    rfn.to(device)

    model = CNN(rfn, INPUT_DIM, EMBEDDING_DIM, N_FILTERS, FILTER_SIZES, OUTPUT_DIM, DROPOUT, PAD_IDX)

    pretrained_embeddings = TEXT.vocab.vectors

    model.embedding.weight.data.copy_(pretrained_embeddings)

    UNK_IDX = TEXT.vocab.stoi[TEXT.unk_token]

    model.embedding.weight.data[UNK_IDX] = torch.zeros(EMBEDDING_DIM)
    model.embedding.weight.data[PAD_IDX] = torch.zeros(EMBEDDING_DIM)
    return model

In [201]:
N_EPOCHS = 5
rfn_mu_vals = [0, .1, .2, .3, .4, .5]

for mu in rfn_mu_vals:
    print(f'Training RFN, mu={mu}:')
    # Set whether RFN will be used in training
    if mu == 0:
        rfn_flag = False
    else:
        rfn_flag = True
    
    print(rfn_flag)
    
    model = load_model(mu, rfn_flag)
    
    optimizer = optim.Adam(model.parameters())

    criterion = nn.BCEWithLogitsLoss()

    model = model.to(device)
    criterion = criterion.to(device)

    best_valid_loss = float('inf')

    for epoch in range(N_EPOCHS):

        start_time = time.time()

        train_loss, train_acc = train(model, train_iterator, optimizer, criterion)
        valid_loss, valid_acc = evaluate(model, valid_iterator, criterion)

        end_time = time.time()

        epoch_mins, epoch_secs = epoch_time(start_time, end_time)

        if valid_loss < best_valid_loss:
            best_valid_loss = valid_loss
            torch.save(model.state_dict(), f'./models/imdb_rfn_{mu}.pt')

        print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
        print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
        print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f}%')

Training RFN, mu=0:
False
Epoch: 01 | Epoch Time: 0m 19s
	Train Loss: 0.501 | Train Acc: 74.73%
	 Val. Loss: 0.336 |  Val. Acc: 86.15%
Epoch: 02 | Epoch Time: 0m 19s
	Train Loss: 0.305 | Train Acc: 87.34%
	 Val. Loss: 0.284 |  Val. Acc: 88.22%
Epoch: 03 | Epoch Time: 0m 19s
	Train Loss: 0.221 | Train Acc: 91.48%
	 Val. Loss: 0.259 |  Val. Acc: 89.22%
Epoch: 04 | Epoch Time: 0m 19s
	Train Loss: 0.148 | Train Acc: 94.82%
	 Val. Loss: 0.277 |  Val. Acc: 89.02%
Epoch: 05 | Epoch Time: 0m 20s
	Train Loss: 0.088 | Train Acc: 97.06%
	 Val. Loss: 0.292 |  Val. Acc: 89.26%
Training RFN, mu=0.1:
True
Epoch: 01 | Epoch Time: 0m 20s
	Train Loss: 0.511 | Train Acc: 73.56%
	 Val. Loss: 0.358 |  Val. Acc: 84.67%
Epoch: 02 | Epoch Time: 0m 18s
	Train Loss: 0.333 | Train Acc: 85.61%
	 Val. Loss: 0.312 |  Val. Acc: 86.53%
Epoch: 03 | Epoch Time: 0m 18s
	Train Loss: 0.260 | Train Acc: 89.66%
	 Val. Loss: 0.278 |  Val. Acc: 88.41%
Epoch: 04 | Epoch Time: 0m 21s
	Train Loss: 0.195 | Train Acc: 92.68%
	 Val

Finally, we train our model...

We get test results comparable to the previous 2 models!

In [206]:
rfn_mu_vals = [0, .1, .2, .3, .4, .5]
for mu in rfn_mu_vals:
    model.load_state_dict(torch.load(f'./models/imdb_rfn_{mu}.pt'))

    test_loss, test_acc = evaluate(model, test_iterator, criterion)

    print(f'MU: {mu} | Test Loss: {test_loss:.3f} | Test Acc: {test_acc*100:.2f}%')

MU: 0 | Test Loss: 0.441 | Test Acc: 79.98%
MU: 0.1 | Test Loss: 0.400 | Test Acc: 82.45%
MU: 0.2 | Test Loss: 0.387 | Test Acc: 82.66%
MU: 0.3 | Test Loss: 0.379 | Test Acc: 83.45%
MU: 0.4 | Test Loss: 0.389 | Test Acc: 82.88%
MU: 0.5 | Test Loss: 0.365 | Test Acc: 83.97%


## User Input

And again, as a sanity check we can check some input sentences

**Note**: As mentioned in the implementation details, the input sentence has to be at least as long as the largest filter height used. We modify our `predict_sentiment` function to also accept a minimum length argument. If the tokenized input sentence is less than `min_len` tokens, we append padding tokens (`<pad>`) to make it `min_len` tokens.

In [225]:
import spacy
nlp = spacy.load('en')

model.load_state_dict(torch.load(f'./models/imdb_rfn_{0.5}.pt'))

def predict_sentiment(model, sentence, min_len = 5):
    model.eval()
    tokenized = [tok.text for tok in nlp.tokenizer(sentence)]
    if len(tokenized) < min_len:
        tokenized += ['<pad>'] * (min_len - len(tokenized))
    indexed = [TEXT.vocab.stoi[t] for t in tokenized]
    tensor = torch.LongTensor(indexed).to(device)
    tensor = tensor.unsqueeze(0)
    prediction = torch.sigmoid(model(tensor))
    return prediction.item()

An example negative review...

In [232]:
predict_sentiment(model, "This film is terrible")

0.1745041310787201

An example positive review...

In [227]:
predict_sentiment(model, "This film is great")

0.7319116592407227

##FSGM Application

In [425]:
def fgsm_batch(net, orig, criterion, eps=15):
    '''Given a pytorch CNN model, original image as tensor, and value for epsilon, create adversarial sample
    using fast gradient sign method (FGSM) described in arXiv:1412.6572v3'''
    # Create input tensor from original image, unsqeezing by a dim to account for model expected batch dim
    # float cast is likely not needed after normalization, but as float is required for cuda execution,
    # it's best to ensure float data
    inp = orig.float().clone().detach().to(device).requires_grad_(True)
    print(inp)
    
    # Make prediction prior to attack to use in calulating gradients
    print('before prior', inp.dtype, inp.size())
    #for sample in inp:
    prior_out = net(inp)
    #print(prior_out.size(), prior_out)
    prior_pred = F.relu(prior_out)
    #print(prior_pred.size(), prior_pred)

    # Use prediction vector to compute loss and then compute gradients from loss to use in attack
    loss = criterion(prior_out, prior_pred)
    loss.backward()
    print(loss.grad_fn)
    # Actual FSGM method
    #print(orig.min(), orig.max())
    print(dir(inp.grad))
    inp = inp + ((eps/255.0) * torch.sign(inp.grad.data))
    inp = inp.clamp(min=orig.min(), max=orig.max())
    #print(inp.min(), inp.max())

    #inp = inp.long().clone().detach().to(device).requires_grad_(False)
    
    return inp

In [426]:
def fgsm_evaluate(model, iterator, criterion, eps=15, adv=False):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.eval()
    
    for batch in iterator:
        if adv:
            print('Before FGSM',batch.text[0].dtype)
            print(batch.label)
            batch.text = fgsm_batch(model, batch.text, criterion, eps = eps)

        with torch.no_grad():
            predictions = model(batch.text).squeeze(1)

            loss = criterion(predictions, batch.label)

            acc = binary_accuracy(predictions, batch.label)

            epoch_loss += loss.item()
            epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [427]:
'''def fsgm_evaluate(net, dataloader, eps=15, adv=False):
    correct = 0
    count = 0
    for i, data in enumerate(dataloader, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data
        inputs = inputs.to(device)
        labels = labels.to(device)
        if adv:
            inputs = fgsm_batch(net, inputs, eps = eps)
        preds = net(inputs)
        if torch.argmax(preds) == labels:
            correct += 1.0
        count += 1.0
        if not i % 5000:
            print(f'Sample: {i}, Running Accuracy: {(correct/count):5.3f}')
    acc = correct/count
    print(f'Accuracy over {int(count)} samples: {acc:5.3f}')
    return acc'''

"def fsgm_evaluate(net, dataloader, eps=15, adv=False):\n    correct = 0\n    count = 0\n    for i, data in enumerate(dataloader, 0):\n        # get the inputs; data is a list of [inputs, labels]\n        inputs, labels = data\n        inputs = inputs.to(device)\n        labels = labels.to(device)\n        if adv:\n            inputs = fgsm_batch(net, inputs, eps = eps)\n        preds = net(inputs)\n        if torch.argmax(preds) == labels:\n            correct += 1.0\n        count += 1.0\n        if not i % 5000:\n            print(f'Sample: {i}, Running Accuracy: {(correct/count):5.3f}')\n    acc = correct/count\n    print(f'Accuracy over {int(count)} samples: {acc:5.3f}')\n    return acc"

In [428]:
from collections import defaultdict
# Create history dict for all models with validation accuracy and loss per epoch
accs_no_fsgm = defaultdict(list)
accs_fsgm = defaultdict(list)

In [429]:
# load IMDB models
#train_dl = DataLoader(trainset, batch_size=1)

rfn_mu_vals = [0, .1, .2, .3, .4, .5]

#epsilons = [10, 20, 30, 40]
epsilons = [40]

for eps in epsilons:
    for adv in [True, False]:
        for i, mu in enumerate(rfn_mu_vals):
            imdb_rfn = load_model(mu=mu, rfn_flag=True)
            imdb_rfn.load_state_dict(torch.load(f'./models/imdb_rfn_{mu}.pt'))

            criterion = nn.BCEWithLogitsLoss()
            imdb_rfn = imdb_rfn.to(device)
            criterion = criterion.to(device)
            
            if adv:
                print(f'Evaluate IMDB RFN at epsilon={eps}, mu={mu}:')
                accs_fsgm[mu] = fgsm_evaluate(imdb_rfn, train_iterator, criterion, eps=eps, adv=adv)
            else:
                print(f'Evaluate IMDB RFN without FSGM, mu={mu}:')
                accs_no_fsgm[mu] = fgsm_evaluate(imdb_rfn, train_iterator, criterion, eps=eps, adv=adv)

Evaluate IMDB RFN at epsilon=40, mu=0:
Before FGSM torch.int64
tensor([1., 0., 0., 1., 0., 1., 1., 0., 0., 0., 0., 1., 0., 1., 0., 1., 0., 0.,
        1., 0., 0., 1., 0., 0., 0., 1., 1., 0., 0., 0., 0., 1., 1., 0., 1., 1.,
        0., 0., 0., 1., 0., 0., 1., 0., 1., 1., 1., 0., 1., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0.], device='cuda:0')
tensor([[4.4290e+03, 1.7000e+01, 1.6850e+03,  ..., 1.8905e+04, 1.8000e+01,
         9.7490e+03],
        [6.6000e+01, 2.3000e+01, 1.9000e+01,  ..., 1.0000e+00, 1.0000e+00,
         1.0000e+00],
        [6.6000e+01, 9.0000e+00, 2.7000e+01,  ..., 1.0000e+00, 1.0000e+00,
         1.0000e+00],
        ...,
        [6.6000e+01, 1.9000e+01, 3.9400e+02,  ..., 1.0000e+00, 1.0000e+00,
         1.0000e+00],
        [0.0000e+00, 2.1000e+01, 3.2400e+02,  ..., 1.0000e+00, 1.0000e+00,
         1.0000e+00],
        [1.5750e+03, 5.9000e+01, 4.1300e+02,  ..., 1.0000e+00, 1.0000e+00,
         1.0000e+00]], device='cuda:0', requires_grad=True

AttributeError: 'NoneType' object has no attribute 'data'

In [245]:
print(f'MU\tIMDB\tFSGM eps=40')

for mu in rfn_mu_vals:
    print(f'{mu}\t{accs_no_fsgm[mu][1]:5.3}\t{accs_no_fsgm[mu][1]:5.3}')

MU	IMDB	FSGM eps=40
0	0.955	0.955
0.1	 0.96	 0.96
0.2	0.915	0.915
0.3	 0.94	 0.94
0.4	 0.92	 0.92
0.5	0.896	0.896


In [244]:
accs_no_fsgm[0]

(0.1455942576166487, 0.955120894160584)

In [257]:
for batch in train_iterator:
    print(type(batch), batch.text[0].size())

<class 'torchtext.data.batch.Batch'> torch.Size([831])
<class 'torchtext.data.batch.Batch'> torch.Size([1111])
<class 'torchtext.data.batch.Batch'> torch.Size([1207])
<class 'torchtext.data.batch.Batch'> torch.Size([1098])
<class 'torchtext.data.batch.Batch'> torch.Size([994])
<class 'torchtext.data.batch.Batch'> torch.Size([1163])
<class 'torchtext.data.batch.Batch'> torch.Size([835])
<class 'torchtext.data.batch.Batch'> torch.Size([1365])
<class 'torchtext.data.batch.Batch'> torch.Size([1078])
<class 'torchtext.data.batch.Batch'> torch.Size([1201])
<class 'torchtext.data.batch.Batch'> torch.Size([720])
<class 'torchtext.data.batch.Batch'> torch.Size([830])
<class 'torchtext.data.batch.Batch'> torch.Size([1094])
<class 'torchtext.data.batch.Batch'> torch.Size([862])
<class 'torchtext.data.batch.Batch'> torch.Size([1179])
<class 'torchtext.data.batch.Batch'> torch.Size([1191])
<class 'torchtext.data.batch.Batch'> torch.Size([1165])
<class 'torchtext.data.batch.Batch'> torch.Size([1122]

KeyboardInterrupt: 